### [ 자동차별 재원 및 리뷰 웹 크롤링 해오기 ]
- 현대, 기아, 제네시스, 르노 / 벤츠, BMW, 테슬라, 렉스턴 8개의 브랜드
- 기간 무의미


In [1]:
## 모듈 로딩
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time


In [2]:
## 옵션 정의
options = Options()
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


In [3]:
driver = webdriver.Chrome(options=options)

cars = ['현대', '기아', '제네시스', '르노', '벤츠', 'BMW', '테슬라', '렉서스']

data_set = set()


In [4]:

for brand in cars:
    
    driver.get("https://www.naver.com")
    time.sleep(2) 

    search_box = driver.find_element(By.CSS_SELECTOR, "input#query")
    search_box.clear()
    search_box.send_keys(f"{brand} 국내전체 자동차")
    search_box.send_keys(Keys.ENTER)

    print(f"🔍 검색 완료: {brand}")

    # JS 로딩 기다리기
    time.sleep(3)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    ## 첫번재 페이지 차이름 가져오기
    temp = soup.select('a._text')
    
    for t in temp :

        data_set.add(t.text)
    total = int(soup.select_one('#main_pack > div.sc_new.cs_common_module.case_list.color_5._list_car > div.cm_content_wrap > div.cm_content_area > div > div.cm_paging_area.no_margin > div > span > span._total').text)
    # print(total)
    
    ## 다음 버튼 객체
    next_box = driver.find_element(By.CSS_SELECTOR, '#main_pack > div.sc_new.cs_common_module.case_list.color_5._list_car > div.cm_content_wrap > div.cm_content_area > div > div.cm_paging_area.no_margin > div > a.pg_next.on')
    
    ## 다음 버튼 누르고 자동차 이름 저장
    for _ in range(total - 1) :
        next_box.send_keys(Keys.ENTER)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        temp = soup.select('a._text')
        for t in temp :
            data_set.add(t.text)
  
    print(len(data_set))

# driver.quit()



🔍 검색 완료: 현대
438
🔍 검색 완료: 기아
865
🔍 검색 완료: 제네시스
953
🔍 검색 완료: 르노
1044
🔍 검색 완료: 벤츠
1478
🔍 검색 완료: BMW
1965
🔍 검색 완료: 테슬라
1989
🔍 검색 완료: 렉서스
2215


In [5]:
print(len(data_set))
# print(data_set)
new_data_set = set()

for s in data_set :
    if '.' not in s : new_data_set.add(s)
    
print(len(new_data_set))
# print(new_data_set)
    

2215
1852


In [6]:
url = r'https://search.naver.com/search.naver?sm=tab_sly.hst&where=nx&ssc=tab.nx.all&query=%EB%A0%89%EC%84%9C%EC%8A%A4+%EA%B5%AD%EB%82%B4%EC%A0%84%EC%B2%B4+%EC%9E%90%EB%8F%99%EC%B0%A8&oquery=%EB%A0%89%EC%84%9C%EC%8A%A4%EA%B5%AD%EB%82%B4%EC%A0%84%EC%B2%B4+%EC%9E%90%EB%8F%99%EC%B0%A8&tqi=jfTfWsqVOswssTSfhMdssssssu8-477995&acq=%EB%A0%89%EC%84%9C%EC%8A%A4+%EA%B5%AD%EB%82%B4%EC%A0%84%EC%B2%B4+%EC%9E%90%EB%8F%99%EC%B0%A8&acr=1&qdt=0&ackey=ldpok2z1'

driver = webdriver.Chrome(options=options)
driver.get(url)
time.sleep(2)

# for s in data_set:
    
#     # 검색창이 페이지 새로 로드된 뒤에도 반드시 새로 찾도록 처리
#     search_box = WebDriverWait(driver, 5).until(
#         EC.presence_of_element_located((By.CSS_SELECTOR, "#nx_query"))
#     )

#     # 기존 텍스트 삭제 방식 변경 (더 안정적)
#     search_box.send_keys(Keys.CONTROL + "a")
#     search_box.send_keys(Keys.DELETE)

#     search_box.send_keys(s)
#     search_box.send_keys(Keys.ENTER)
    
    

#     print(f" 검색 성공 → {s}")
#     time.sleep(1.5)

final_specs_list = []

for idx, car_name in enumerate(new_data_set):
    print(f"[{idx+1}/{len(data_set)}] ↓ '{car_name}' 제원 검색 중 ..")

    try:
        # driver.get("https://www.naver.com")
        time.sleep(1)

        search_box = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#nx_query"))
        )

        search_box.send_keys(Keys.CONTROL + "a")
        search_box.send_keys(Keys.DELETE)
        search_box.send_keys(f"{car_name} 제원")
        search_box.send_keys(Keys.ENTER)

        time.sleep(2)

        # 제원표 요소 기다리기
        try:
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, "table_group"))
            )
        except:
            print("   → 제원표 없음 | 스킵")
            continue

        soup = BeautifulSoup(driver.page_source, "html.parser")

        # 불필요 태그 제거
        for t in soup.select(".layer_info, .btn_info, .ico_info, .blind"):
            t.decompose()

        car_spec = {"차종": car_name}

        for group in soup.select("div.table_group"):
            rows = group.select("table.basic_table tr")
            for row in rows:
                try:
                    key = row.select_one("th").get_text(strip=True)
                    val = row.select_one("td").get_text(strip=True)
                    car_spec[key] = val
                except:
                    pass

        final_specs_list.append(car_spec)
        print(f"   → 수집 완료 ({len(car_spec)} 항목)")

        # time.sleep(random.uniform(1.0, 2.3))

    except Exception as e:
        print(f" 에러 발생: {e}")
        continue

[1/2215] ↓ '2005 그랜저' 제원 검색 중 ..
   → 수집 완료 (22 항목)
[2/2215] ↓ '2021 쏘나타' 제원 검색 중 ..
 에러 발생: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=142.0.7444.176)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff72c02a235
	0x7ff72bd82630
	0x7ff72bb116dd
	0x7ff72bae8e31
	0x7ff72bb9a22e
	0x7ff72bbbb3d2
	0x7ff72bb5b0ed
	0x7ff72bb5bf63
	0x7ff72c055d60
	0x7ff72c04fe8a
	0x7ff72c071005
	0x7ff72bd9d71e
	0x7ff72bda4e1f
	0x7ff72bd8b7c4
	0x7ff72bd8b97f
	0x7ff72bd718e8
	0x7ffbd19ee8d7
	0x7ffbd292c53c

[3/2215] ↓ '2026 BMW 4시리즈' 제원 검색 중 ..
 에러 발생: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=142.0.7444.176)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff72c02a235
	0x7ff72bd82630
	0x7ff72bb116dd
	0x7ff72bae8e31
	0x7ff72bb9a22e
	0x7ff72bbbb3d2
	0x7ff72bb5b0ed
	0x7ff72bb5bf63
	0x7ff72c055d60
	0x7ff72c04fe8a
	0x7ff72c071005


KeyboardInterrupt: 

In [ ]:
final_reviews_list = []
car_review = {}

for idx, car_name in enumerate(new_data_set):
    print(f"[{idx+1}/{len(new_data_set)}] ⬇ '{car_name}' 평가 검색 중...")

    try:
        time.sleep(1)

        search_box = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#nx_query"))
        )

        search_box.send_keys(Keys.CONTROL + "a")
        search_box.send_keys(Keys.DELETE)
        search_box.send_keys(f"{car_name} 평가")  # ← 여기만 변경!
        search_box.send_keys(Keys.ENTER)

        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        # 평가 데이터 수집 (구조 파악 필요)
        car_review = {"차종": car_name}
        ## 차량 풀네임 넣기
        full_name = soup.select_one('#root > section > div.top_wrap > div.top_area > div > h2 > strong').text
        print(full_name)
        car_review['풀네임'] = full_name


       

        # 예상되는 선택자 (실제로는 확인 필요)
        # 1. 항목별 점수 영역 찾기
        score_items = soup.select("li.chart_label_item")  # 또는 다른 selector
        
        for item in score_items:
            try:
                # 점수 (9.5 같은 숫자)
                score = item.select_one("span.num").get_text(strip=True)
                # 카테고리명 (주행, 연비, 디자인 등)
                category = item.get_text(strip=True)
                category = category[:-len(score)]
                
                car_review[category] = score
            except:
                pass

        # 2. 종합 점수 (text 38.11 x 26 같은 부분)
        try:
            total_score = soup.select_one("#root > section > div.content_wrap.cs_common_module > div > div > div.scroll_box.rating_box > div > div > ul > li:nth-child(1) > div > div.rating_area > span.rating_score").get_text(strip=True)
            car_review["종합점수"] = total_score
        except:
            pass

        if len(car_review) > 1:  # 차종 외에 데이터가 있으면
            
            final_reviews_list.append(car_review)
            print(car_review)
            print(f"   → 수집 완료 ({len(car_review)} 항목)")

        else:
            print("   → 평가 데이터 없음 | 스킵")

    except Exception as e:
        print(f"   ⚠ 에러 발생: {e}")
        continue

driver.quit()

[1/1907] ⬇ '2024 BMW X3 M' 평가 검색 중...
2024 BMW X3 M
{'차종': '2024 BMW X3 M', '풀네임': '2024 BMW X3 M', '주행': '9.8', '가격': '8.3', '거주성': '8.9', '품질': '8.8', '디자인': '9.8', '연비': '4.2', '종합점수': '8.3'}
   → 수집 완료 (9 항목)
[2/1907] ⬇ '2023 싼타페' 평가 검색 중...
2023 현대 싼타페
{'차종': '2023 싼타페', '풀네임': '2023 현대 싼타페', '주행': '9.3', '가격': '8.3', '거주성': '9.1', '품질': '8.7', '디자인': '8.6', '연비': '7.5', '종합점수': '8.6'}
   → 수집 완료 (9 항목)
[3/1907] ⬇ '2014 에쿠스 리무진' 평가 검색 중...
2014 현대 에쿠스 리무진
{'차종': '2014 에쿠스 리무진', '풀네임': '2014 현대 에쿠스 리무진', '주행': '9.4', '가격': '8.8', '거주성': '9.4', '품질': '9.2', '디자인': '9.4', '연비': '6.9', '종합점수': '8.9'}
   → 수집 완료 (9 항목)
[4/1907] ⬇ '2026 카니발' 평가 검색 중...
2026 기아 카니발
{'차종': '2026 카니발', '풀네임': '2026 기아 카니발', '주행': '9.2', '가격': '7.9', '거주성': '9.4', '품질': '8.7', '디자인': '9.5', '연비': '7.3', '종합점수': '8.7'}
   → 수집 완료 (9 항목)
[5/1907] ⬇ '2019 아이오닉 일렉트릭' 평가 검색 중...
2019 현대 아이오닉 일렉트릭
{'차종': '2019 아이오닉 일렉트릭', '풀네임': '2019 현대 아이오닉 일렉트릭', '주행': '9.2', '가격': '9', '거주성': '8.3', '품질': '8.5', '디자인': '7.8',

KeyboardInterrupt: 

In [ ]:
# final_reviews_list DF로 만들기
import pandas as pd
DF = pd.DataFrame(final_reviews_list)
DF

# CSV로 저장
DF.to_csv("review.csv", index=False)


In [ ]:
print(final_reviews_list)

[{'차종': '2025 코나 일렉트릭', '주행': '9.8', '가격': '7.7', '거주성': '9.4', '품질': '9.4', '디자인': '9.6', '연비': '8.7', '종합점수': '9.1'}, {'차종': '2020 G70', '주행': '9.5', '가격': '8.1', '거주성': '7.3', '품질': '8.9', '디자인': '9.6', '연비': '5.9', '종합점수': '8.2'}, {'차종': '2015 메르세데스-벤츠 SL', '주행': '9.6', '가격': '8.2', '거주성': '8.5', '품질': '9.7', '디': '9', '연비': '7.6', '종합점수': '8.8'}, {'차종': '2017 BMW X6', '주행': '9.8', '가격': '8.4', '거주성': '8.6', '품질': '9.4', '디자인': '9.8', '연비': '8.9', '종합점수': '9.1'}, {'차종': '2020 G90L', '주행': '9.4', '가격': '7.9', '거': '9', '품질': '8.9', '디자인': '9.1', '연비': '6.5', '종합점수': '8.5'}, {'차종': '2022 테슬라 모델 Y', '주행': '9.8', '가격': '8.9', '거주성': '9.5', '품질': '8.7', '디자인': '9.5', '연비': '9.4', '종합점수': '9.3'}, {'차종': '1991 엑셀', '주': '10', '가': '10', '거주': '10', '품': '10', '디자': '10', '연': '10', '종합점수': '10'}, {'차종': '1990 엑셀 해치백', '주': '10', '가': '10', '거주': '10', '품': '10', '디자': '10', '연': '10', '종합점수': '10'}, {'차종': '2018 i40 ', '': '8', '거': '9', '품질': '8.8', '디자인': '9.1', '연비': '8.6', '종합점수': '8.